In [141]:
import pandas as p
import os
import sys
import glob
import numpy as np
sys.path.insert(0, '../')

In [114]:
np.sqrt(.3)

0.5477225575051661

### Import data for one stock

In [115]:
ph_df = pd.read_csv('../data/price-volume-data-for-all-us-stocks-etfs/Stocks/ph.us.txt')

In [116]:
ph_df[ph_df.Date == '2017-01-05']

,Date,Open,High,Low,Close,Volume,OpenInt
7944,2017-01-05,140.04,140.35,137.65,139.07,567140,0


In [117]:
stock_to_indsutry

,symbol,industry
0,krus,restaurant
1,blmn,restaurant
2,frgi,restaurant
3,ruth,restaurant
4,eat,restaurant
...,...,...
195,rop,industrial conglomerates
196,rost,retail
197,so,utilities
198,ual,airlines


### Run through stocks we pulled data from IEX for. Update the .csv file with this historical data

In [145]:
def add_historic_data_to_current_csv():
#     current_data_file_path = '../data/historical_etf_data'
    current_data_file_path = '../data/historical_stock_data'
    augmented_data_file_path = '../data/updated_historical_stock_and_etf_data'
#     historic_data_file_path = '../data/price-volume-data-for-all-us-stocks-etfs/ETFs'
    historic_data_file_path = '../data/price-volume-data-for-all-us-stocks-etfs/Stocks'    

    for file in glob.glob(f'{current_data_file_path}/*.csv'):
        print(f'File = {file}')
        stock_name = file.rsplit("/")[-1].split('_')[0].lower() # note: store all stock names as uppercase
        
        # find the stock name in historical kaggle dataa
        for historical_file in glob.glob(f'{historic_data_file_path}/*.txt'):
            historical_stock_name = historical_file.split("/")[-1].split(".")[0]
            
            if stock_name == historical_stock_name:
                print('found historical stock', historical_stock_name)
                # read in current data
                current_stock_data = pd.read_csv(f"{file}")
                print(historical_file, 'historical_file')
                historical_stock_data = pd.read_csv(f"{historical_file}")
                historical_stock_data.columns = map(str.lower, historical_stock_data.columns)

                oldest_date_of_new_data = current_stock_data.date.min()

                # get the new data we don't have yet
                historical_stock_data = historical_stock_data[historical_stock_data.date < oldest_date_of_new_data]
                historical_stock_data.drop('openint', inplace=True, axis=1)


                # concat historical with current data
                combined_stock_df = pd.concat([current_stock_data, historical_stock_data])
                most_recent_date = combined_stock_df.date.max()
                most_recent_year, most_recent_month, most_recent_day = most_recent_date.split('-')

                oldest_date = combined_stock_df.date.min()
                oldest_year, oldest_month, oldest_day = oldest_date.split('-')
                
               # delete the old file only if we haven't  updated it
                delete_old_csv(stock_name, augmented_data_file_path)
                
                # save the file
                combined_stock_df.to_csv(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv',
                                         index=False)
             
                break

        # # if we don't have historical data , save the original data into the augmented folder
        # if os.path.isfile(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv'):
        #     pass
        # else:
        #     current_stock_data = pd.read_csv(f"{file}")
        #     current_stock_data.to_csv(f"{augmented_data_file_path}/)



In [146]:
def delete_old_csv(stock_name, folder_path):
    for file in glob.glob(f'{folder_path}/*.csv'):
        if stock_name in file:
            os.remove(file)

In [147]:
add_historic_data_to_current_csv()

File = ../data/historical_stock_data/urbn_2017-01-03_to_2020-5-1.csv
found historical stock urbn
../data/price-volume-data-for-all-us-stocks-etfs/Stocks/urbn.us.txt historical_file
File = ../data/historical_stock_data/STT_2017-01-03_to_2020-5-8.csv
found historical stock stt
../data/price-volume-data-for-all-us-stocks-etfs/Stocks/stt.us.txt historical_file
File = ../data/historical_stock_data/BOH_2017-1-1_to_2020-5-8.csv
found historical stock boh
../data/price-volume-data-for-all-us-stocks-etfs/Stocks/boh.us.txt historical_file
File = ../data/historical_stock_data/dds_2017-01-03_to_2020-5-1.csv
found historical stock dds
../data/price-volume-data-for-all-us-stocks-etfs/Stocks/dds.us.txt historical_file
File = ../data/historical_stock_data/jwn_2017-01-03_to_2020-5-1.csv
found historical stock jwn
../data/price-volume-data-for-all-us-stocks-etfs/Stocks/jwn.us.txt historical_file
File = ../data/historical_stock_data/CVCY_2017-1-1_to_2020-5-8.csv
found historical stock cvcy
../data/price-